In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import math
import sys


# Get Links From List

In [ ]:

url = "https://en.wikipedia.org/wiki/List_of_computer_scientists"


response = requests.get(url)
first_links_list = []

if response.status_code == 200:
    
 soup = BeautifulSoup(response.text, 'html.parser')

    
 h2_element = soup.find('div', {'class': 'mw-content-ltr mw-parser-output'})
 list_test=h2_element.find_all('ul')

 for ul_element in list_test:
      li_elements = ul_element.find_all('li')
      for li in li_elements:
            link = li.find('a')
            if link:
                href = link.get('href')
                if href:
                    full_link = f"https://en.wikipedia.org{href}"
                    first_links_list.append(full_link)
 links_to_exclude = []
 see_also_header = soup.find('span',id='See_also').parent
 see_also_div = see_also_header.find_next('div')
 see_also_ul = see_also_div.find_all('ul')
 for ul_element in see_also_ul:
      see_also_li_elements = ul_element.find_all('li')
      for li in see_also_li_elements:
        link = li.find('a')
        if link:
            href = link.get('href')
            if href:
                full_link = f"https://en.wikipedia.org{href}"
                links_to_exclude.append(full_link)
first_links_list = [link for link in first_links_list if link not in links_to_exclude]
print(first_links_list)


### Get name

In [8]:
def get_scientist_name(link):
    response= requests.get(link)
    if response.status_code == 200:
    
        soup = BeautifulSoup(response.text, 'html.parser')
        name=soup.find('span',{'class': 'mw-page-title-main'}).text
        if name:
            return name
        else:
            return 'unknown'       

### Get Awards

In [9]:
def get_scientist_awards(link):
    response= requests.get(link)
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.text, 'html.parser')
        find_awards=0
        awards_table = soup.find('table',{'class': 'infobox biography vcard'})
        if(awards_table):
            awards_table_condition = awards_table.find('th', string= 'Awards')
            if awards_table_condition:
                awards_table_td = awards_table_condition.find_next('td',{'class' : 'infobox-data'})
                awards_table_list = awards_table_td.find_next('ul')
                awards=len(awards_table_list.find_all('li'))
                find_awards=1

        if find_awards==0:
            awards_header_condition = soup.find('span', {'id': 'Honors_&_Awards'})
            if awards_header_condition:
                awards_header = soup.find('span', {'id': 'Honors_&_Awards'}).parent
                awards_list=awards_header.find_next('ul')
                li_elements = list(awards_list.find_all('li'))
                awards=len(li_elements)
                find_awards=1
        
        if find_awards==1:
             return awards
        else:
             return 0               

### Get education

In [10]:
def get_scientist_education(link):
    response= requests.get(link)
    if response.status_code == 200:
    
        soup = BeautifulSoup(response.text, 'html.parser')
        find_education=0
        education_table = soup.find('table',{'class': 'infobox biography vcard'})
        if education_table:
            education_table_condition = education_table.find('th', string= 'Education')
            if education_table_condition:
                education_header= education_table_condition.parent
                education_table_td = education_header.find_next('td')
                education_list = education_table_td.find_next('> ul')
                if education_list:
                    education_text = ', '.join(li.get_text(strip=True) for li in ul_element.find_all('li'))
                    find_education=1
                education_a = education_table_td.find_all('a')
                if education_a and find_education==0:
                    education_text = ', '.join(a.get_text(strip=True) for a in education_table_td.find_all('a'))
                    find_education=1
            if find_education==0:
                alma_matter_condition = education_table.find('th', string= 'Alma mater')
                if alma_matter_condition:
                    alma_matter_header = alma_matter_condition.parent
                    alma_matter_td = alma_matter_header.find_next('td')
                    alma_matter_list = alma_matter_td.find_next('> ul')
                    if alma_matter_list:
                        education_text = ', '.join(li.get_text(strip=True) for li in ul_element.find_all('li'))
                        find_education=1

                    alma_matter_a = alma_matter_td.find_all('a')
                    if alma_matter_a and find_education==0 :
                        education_text = ', '.join(a.get_text(strip=True) for a in alma_matter_td.find_all('a'))
                        find_education=1

        if find_education==1:
            return education_text
        else:
             return 'NO_SAVE'

### Into the CSV

In [11]:
# sv_filename = 'scientists_data.csv'

# # Headers
# headers = ['Name', 'Awards', 'Education']

# with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    
#    csv_writer = csv.writer(csv_file)

#    # Write the headers 
#    csv_writer.writerow(headers)

#    #write each scientist's information to the CSV file
#    for link in first_links_list:
#        name = get_scientist_name(link)
#        awards = get_scientist_awards(link)
#        education = get_scientist_education(link)
      

#        #Write the information
#        if education!='NO_SAVE' and not education.startswith('(') and not education.startswith('['):
#            csv_writer.writerow([name, awards, education])



### Clean Garbage

In [12]:
csv_file_path = 'scientists_data.csv'
df = pd.read_csv(csv_file_path)
all_institutes=[]
for row in df['Education']:
    institutes = [inst.strip() for inst in row.split(',')]
    all_institutes.extend(institutes)

unique_institutes = list(set(all_institutes))
unique_institutes.remove('Ph.D')
unique_institutes.remove('B.S.')
unique_institutes.remove('[11]')
unique_institutes.remove('[2]')
unique_institutes.remove('S.B.')
unique_institutes.remove('BSc.')
unique_institutes.remove('es')
unique_institutes.remove('BE')
unique_institutes.remove('[1]')
unique_institutes.remove('[3]')
unique_institutes.remove('[5]')
unique_institutes.remove('PhD')
unique_institutes.remove('[4]')
unique_institutes.remove('MSc')
unique_institutes.remove('B.Sc.')
unique_institutes.remove('BSc')
unique_institutes.remove('BASc')
unique_institutes.remove('Ph.D.')
unique_institutes.remove('1969')
unique_institutes.remove('MS')
unique_institutes.remove('M.S.')
unique_institutes.remove('SM')
unique_institutes.remove('BS')
unique_institutes.remove('SB')
unique_institutes.remove('BA')
unique_institutes.remove('AB')
unique_institutes.remove('B.A.')
unique_institutes.remove('Diplom')
unique_institutes.remove('')
print(df)

                    Name  Awards   
0    Atta ur Rehman Khan      10  \
1         Scott Aaronson       4   
2           Rediet Abebe       3   
3            Hal Abelson       1   
4        Serge Abiteboul       4   
..                   ...     ...   
504       Stanley Zdonik       0   
505        Hussein Zedan       0   
506   Shlomo Zilberstein       0   
507       Jill Zimmerman       0   
508          Konrad Zuse       5   

                                             Education  
0             University of Malaya, COMSATS University  
1    Cornell University, University of California, ...  
2    Cornell University, University of Cambridge, H...  
3    Princeton University, Massachusetts Institute ...  
4                    University of Southern California  
..                                                 ...  
504              Massachusetts Institute of Technology  
505                              University of Bristol  
506  University of California, Berkeley, Technion –...

### Node Class

In [13]:
class peer_node:
    def __init__(self,df,node_num):
        self.values=df
        self.node_num=node_num
        self.predecessor=0
        self.fingertable = pd.DataFrame(columns=['successors', 'index_of_peers'])
        self.key=[]
    def set_key(self,key):
        self.key=key
    def get_key(self,key):
        self.key.append(key)
    def merge_key_list(self,node2):
        self.key=self.key+node2.key
    def set_finger(self,succ,index):
        self.fingertable['successors'] = succ
        self.fingertable['index_of_peers'] = index


### Chord Cirlce Class

In [79]:
class Chord_circle:
    def __init__(self,institutes,df):
        self.institutes=institutes
        self.df=df
        self.max_node=0
        self.peers=[]
        self.fake_hash_table=pd.DataFrame(columns=['number_of_key','key','search_key'])
    def remove_finger_tables(self):
        for peer in self.peers:
            peer.fingertable=pd.DataFrame(columns=['successors', 'index_of_peers'])
    def create_finger_table_for_all(self):
        length=len(self.peers)
        m=math.ceil(math.log2(length))
        temp_start=0
        for peer in self.peers:
            temp_table=[]
            index_table=[]
            first_attempt=0
            for i in range(1,m+1):
                ok=0
                start=peer.node_num+2**(i-1)
                #if peer.node_num==405:
                    #print("FINGERTABLE-->"+ str(start)+ " WITH MAXNODE--> "+str(self.max_node)) #DEBUGGING
                if start > self.max_node:
                    start= start - self.max_node -1
                if (self.return_to_start(peer.node_num)) & (first_attempt==0):
                    #print("THIS IS NODE_NUM--> "+str(peer.node_num)) #DEBUGGING
                    start=0
                    first_attempt=1
                while(ok==0):
                    for index,s_peer in enumerate(self.peers):
                        if s_peer.node_num==start:
                            temp_table.append(start)
                            index_table.append(index)
                            ok=1
                    start+=1
               
            peer.set_finger(temp_table,index_table) 
    def renew_max_node(self):
        self.max_node=int(self.peers[len(self.peers)-1].node_num)
        #print("TYPE OF MAX NODE -->"+str(self.max_node))
        #print("NEW MAX NODE --> "+str(self.max_node)) DEBUGGING
    def create_list_of_peers(self):
        counter=0

        for institute in self.institutes:
            result_rows = []
            self.fake_hash_table.loc[len(self.fake_hash_table)] = {'number_of_key': counter, 'key': institute,'search_key': counter}

            for index, row in self.df.iterrows():
        
                if institute in row['Education']:
                    result_rows.append(row.drop('Education'))


            result_df = pd.DataFrame(result_rows).reset_index(drop=True)
            result_df['institute'] = institute
            peer=peer_node(result_df,counter)
            counter+=1
            peer.get_key(institute)
            self.peers.append(peer)
        self.renew_max_node()
        self.create_finger_table_for_all()
    def return_to_start(self,i):
        #print("MAX="+str(self.max_node)) DEBUGGING
        if (i != 0) & ((i %  self.max_node) == 0):
            return 1 
        else:
            return 0
    def key_to_number(self,lookup_key):
        get_num=None
        for index,row in self.fake_hash_table.iterrows():
            if row['key'].lower()==lookup_key.lower():
                get_num=row['search_key']
                #print('SEARCH KEY IS --->'+str(get_num)) DEBUGGING
                break
        if get_num==None:
            print("No archive for this Institute")
        return get_num
        #print(get_num) DEBUGGING
    def lookup(self,key,threshold):
        node_to_search=self.key_to_number(key)
        print(f"The key is --> {node_to_search}")
        #print('I WANT NODE'+str(node_to_search)) DEBUGGING
        if node_to_search!=None:
            res_df=pd.DataFrame()
            this_node=self.peers[0]
            while(this_node.node_num!=node_to_search):
                print(f"Searching {this_node.node_num} fingertable")
                ft=this_node.fingertable['successors']
                print(f"{ft}") #DEBUGGING
                for index,row in this_node.fingertable.iterrows():
                    if row['successors']==node_to_search:
                        #print('1st if-->'+str(row['successors']))  #DEBBUGING
                            index=row['index_of_peers']
                            this_node=self.peers[index]
                            break
                        
                    elif row['successors']>node_to_search :
                        #print('2nd if') #DEBUGGING
                        this_node_index = index - 1
                        if this_node_index >= 0:
                            previous_row = this_node.fingertable.at[this_node_index, 'index_of_peers']
                            print_row = this_node.fingertable.at[this_node_index, 'successors']
                            print(f'Previous_row = {print_row}')
                            this_node=self.peers[previous_row]
                            break
                    else:
                        #print('3rd if') #DEBUGGING
                        if index == this_node.fingertable.index[-1]:
                            print_this=this_node.fingertable['successors'].max()
                            print(f'Max row = {print_this}')
                            correct_index=this_node.fingertable['index_of_peers'].max()
                            this_node=self.peers[correct_index]
                        else: continue
               
            res_df=this_node.values
            filtered_rows = res_df[(res_df['Awards'] >= threshold) & (res_df['institute']==key)]
            print(filtered_rows)
    def leave_circle(self,node):
        big=1
        for peer in self.peers:
            if peer.node_num==node:
                delete_this_node=peer
                big=0
        if big==1:
            print('There is no such node')
        elif node<=self.max_node:
            #if self.return_to_start(node):
                nn = delete_this_node.fingertable['index_of_peers'].iloc[0]
                print(f'NN is -->{nn}')
                next_node=self.peers[nn]
                next_node.values=pd.concat([next_node.values,delete_this_node.values],ignore_index=True)
                next_node.merge_key_list(delete_this_node)
                self.fake_hash_table.loc[self.fake_hash_table['search_key']==node,'search_key']=next_node.node_num                    
        
                self.peers.remove(delete_this_node)
                self.renew_max_node()
                self.remove_finger_tables()
                self.create_finger_table_for_all()
    def show(self):
        for peer in self.peers:
            key_str = ', '.join(map(str, peer.key))
            print(str(peer.node_num) + ' --> ' + key_str)
    def join_circle(self):
        empty_slots=[]
        last=0
        max_empty=self.fake_hash_table['number_of_key'].iloc[-1]
        stop=max_empty+1
        if max_empty==self.max_node:
            stop=stop+1
        for i in range(0,stop):
            if i>self.max_node:
                empty_slots.append(i)
            elif i<len(self.peers)-1:
                #print(self.peers[i].node_num) DEBUGGING
                #print(f'Max node is {self.max_node} and i is {i+1}')
                D=self.peers[i+1].node_num-self.peers[i].node_num
                if D>1 : 
                    for j in range(self.peers[i].node_num+1,self.peers[i+1].node_num):
                        empty_slots.append(j)
        print("Choose a slot from these:")
        for slot in empty_slots:
            print(slot)
        sys.stdout.flush() 
        chosen_node=int(input("Enter here ---> "))
        print(chosen_node)
        if chosen_node not in empty_slots:
            print("Thats not a valid node!")
        elif chosen_node==max(empty_slots) &  self.fake_hash_table['number_of_key'].iloc[-1]!=chosen_node:
            empty_df=pd.DataFrame()
            peer_n=peer_node(empty_df,chosen_node)
            self.peers.append(peer_n)
        else:
            new_keys=[]
            new_node_df=pd.DataFrame()
            pre=chosen_node-1
            while pre in empty_slots and pre>=0:
                pre=pre-1
            if chosen_node==max(empty_slots):
                next_node_num=self.peers[0].node_num
                last=1
            else:
                suc=chosen_node+1
                while suc in empty_slots:
                    suc=suc+1
                next_node_num=suc
                while pre in empty_slots and pre>=0:
                    pre=pre-1
                    if pre<0:
                        pre=self.max_node
                
            self.fake_hash_table.loc[(self.fake_hash_table['number_of_key'] <= chosen_node) & (self.fake_hash_table['number_of_key'] >pre), 'search_key'] = chosen_node
            condition=(self.fake_hash_table['search_key'] <= chosen_node) & (self.fake_hash_table['search_key'] >pre)
            selected_rows = self.fake_hash_table[condition]
            print(selected_rows)
            list_try=selected_rows['key'].tolist()
            for peer in self.peers:
                if peer.node_num==next_node_num:
                    split_condition = peer.values['institute'].isin(list_try)
                    new_node_df=peer.values[split_condition]
                    peer.values=peer.values[~split_condition]
                    peer_index=self.peers.index(peer)
                    for key in list_try:
                        new_keys.append(key)
                        peer.key.remove(key)
                    #print(peer.values)
            # self.peers(next_node).value
            peer_n=peer_node(new_node_df,chosen_node)
            peer_n.set_key(new_keys)
            if last==1:
                print(new_node_df)
                self.peers.append(peer_n)
            else:
                self.peers.insert(peer_index,peer_n)
        self.renew_max_node()
        self.remove_finger_tables()
        self.create_finger_table_for_all()

## Δημιουργία αντικειμένου
### Δημιουργία Chord Circle

In [80]:
Chord=Chord_circle(unique_institutes,df)
Chord.create_list_of_peers()

## Εμφάνιση κύκλου

In [ ]:
Chord.show()

## Εμφάνιση fake_hash_table

In [82]:
Fake_has_table=Chord.fake_hash_table
print(Fake_has_table)

     number_of_key                                  key  search_key
0                0                  Harvey Mudd College           0
1                1       Indian Institute of Technology           1
2                2                     Brock University           2
3                3                       Rhodes College           3
4                4                University of Alberta           4
..             ...                                  ...         ...
402            402           Instituto Superior Técnico         402
403            403        Institute of Control Sciences         403
404            404  University of California - Berkeley         404
405            405                 University of Lisbon         405
406            406              University of Cambridge         406

[407 rows x 3 columns]


## Leave Circle κάποιων κόμβων

In [83]:
Chord.leave_circle(11)
Chord.leave_circle(12)
Chord.leave_circle(13)
Chord.leave_circle(311)
Chord.leave_circle(88)
Chord.leave_circle(173)
Chord.leave_circle(389)
Chord.leave_circle(404)
Chord.leave_circle(405)
Chord.leave_circle(406)

NN is -->12
NN is -->12
NN is -->12
NN is -->309
NN is -->86
NN is -->170
NN is -->384
NN is -->398
NN is -->398
NN is -->0


## Join Circle κάποιων κόμβων

In [ ]:
Chord.join_circle()

## LookUps

In [87]:
Chord.lookup("University of Cambridge",0)

The key is --> 406
Searching 0 fingertable
0      1
1      2
2      4
3      8
4     16
5     32
6     64
7    128
8    256
Name: successors, dtype: int64
Max row = 256
Searching 256 fingertable
0    257
1    258
2    260
3    264
4    272
5    288
6    320
7    384
8    105
Name: successors, dtype: int64
Max row = 384
Searching 384 fingertable
0    385
1    386
2    388
3    392
4    400
5      9
6     41
7    105
8    233
Name: successors, dtype: int64
Max row = 400
Searching 400 fingertable
0    401
1    402
2    406
3      1
4      9
5     25
6     57
7    121
8    249
Name: successors, dtype: int64
                                    Name  Awards                institute
1                           Rediet Abebe       3  University of Cambridge
2                       Rosemary Candlin       0  University of Cambridge
3                    Christopher J. Date       0  University of Cambridge
4               Charlotte Froese Fischer       4  University of Cambridge
5                  